In [1]:
import numpy as np

data=np.load('data.npy')
target=np.load('target.npy')

In [2]:
from sklearn.preprocessing import MinMaxScaler

data_scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=data_scaler.fit_transform(data)

target_scaler=MinMaxScaler(feature_range=(0,1))
scaled_target=target_scaler.fit_transform(target)

In [3]:
import joblib

joblib.dump(data_scaler,'data_scaler.sav')
joblib.dump(target_scaler,'target_scaler.sav')

['target_scaler.sav']

In [4]:
scaled_data=scaled_data.reshape(scaled_data.shape[0],scaled_data.shape[1],1)
scaled_target=scaled_target.reshape(scaled_target.shape[0],scaled_target.shape[1],1)

In [5]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(scaled_data,scaled_target,test_size=0.2)

In [6]:
from keras.layers import Input,Dense,Conv1D,MaxPool1D,UpSampling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint


input_layer=Input(shape=scaled_data.shape[1:])  

model=Conv1D(32,(3),activation='relu',padding='same')(input_layer)
model=MaxPool1D((2),padding='same')(model)
model=Conv1D(32,(3),activation='relu',padding='same')(model)
encoder=MaxPool1D((2),padding='same')(model)

model=Conv1D(32,(3),activation='relu',padding='same')(encoder)
model=UpSampling1D((2))(model) 
model=Conv1D(32,(3),activation='relu',padding='same')(model)
model=UpSampling1D((2))(model) 

decoder=Conv1D(1,(3),activation='relu',padding='same')(model)

autoencoder=Model(input_layer,decoder)

autoencoder.compile(optimizer='adam',loss='mse')

In [7]:
checkpoint = ModelCheckpoint('models/model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
autoencoder.fit(train_data,train_target,epochs=100,validation_data=(test_data, test_target),callbacks=[checkpoint])

150/150 [==============================] - 17s 70ms/step - loss: 0.0124 - val_loss: 0.0102
INFO:tensorflow:Assets written to: models\model-001.model\assets


In [8]:
np.save('test_data',test_data)
np.save('test_target',test_target)